## 1. Setup & Imports

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Add project to path
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

# Import from libs
from libs.file_operations import (
    movePreprocessed,
    freemove,
    move2preprocess,
    move2convert,
    moveConverted,
)
from libs.metadata import (
    createMetaCombinedString,
    exportCSV,
    filterMetadata,
)
from libs.utils import (
    validate_directory_structure,
    ensure_output_directories,
    print_pipeline_status,
    list_available_metadata,
    get_directory_summary,
)
from libs.config import (
    BASE_DIR,
    SEQUENCES,
    CONDITIONS,
    TEMP_META_DIR,
)

print("✓ All imports successful!")
print(f"Project root: {project_root}")

## 2. Pre-flight Check

Verify that your ADNI data directory structure is set up correctly.

In [ ]:
# Create output directories
ensure_output_directories(str(BASE_DIR))
print("\n✓ Output directories ensured!\n")

# Validate directory structure
validation = validate_directory_structure(str(BASE_DIR))
print("Directory Structure Validation:")
for name, exists in validation.items():
    status = "✓" if exists else "✗"
    print(f"  {status} {name}")

## 3. Pipeline Status Dashboard

Check current state of all data directories.

In [ ]:
# Get summary statistics
summary = get_directory_summary(str(BASE_DIR))

print("\n" + "="*70)
print("ADNI Pipeline Status")
print("="*70)
print("\nFile Counts by Directory:")
for name, count in summary.items():
    print(f"  {name}: {count}")

# Available metadata
meta_files = list_available_metadata(str(TEMP_META_DIR))
print("\nAvailable Metadata Files:")
if meta_files:
    for f in meta_files:
        print(f"  • {f}")
else:
    print("  (No metadata files found)")
print("="*70 + "\n")

## 4. Load Metadata

Select and load metadata CSV file for your workflow.

In [ ]:
# Get available metadata files
available_meta = list_available_metadata(str(TEMP_META_DIR))

if not available_meta:
    print("⚠ No metadata files found!")
    print(f"\nPlace your metadata CSV files in: {TEMP_META_DIR}")
    print("\nExpected columns:")
    print("  Image Data ID, Subject, Group, Sex, Age, Visit, Modality, Description, Type, Acq Date, Format")
else:
    # Create dropdown for metadata selection
    meta_dropdown = widgets.Dropdown(
        options=available_meta,
        description='Metadata File:',
        style={'description_width': '120px'}
    )
    
    load_button = widgets.Button(description='Load Metadata', button_style='info')
    output = widgets.Output()
    
    def on_load_clicked(b):
        with output:
            clear_output()
            try:
                meta_path = TEMP_META_DIR / meta_dropdown.value
                global meta_df
                meta_df = pd.read_csv(meta_path)
                print(f"✓ Loaded: {meta_dropdown.value}")
                print(f"  Shape: {meta_df.shape}")
                print(f"\nFirst few rows:")
                display(meta_df.head())
            except Exception as e:
                print(f"✗ Error loading metadata: {e}")
    
    load_button.on_click(on_load_clicked)
    
    display(widgets.HBox([meta_dropdown, load_button]))
    display(output)

## 5. Interactive Parameter Selection

Choose workflow parameters using dropdowns.

In [ ]:
# Create interactive parameter selectors
seq_widget = widgets.Dropdown(
    options=SEQUENCES,
    description='Sequence:',
    style={'description_width': '100px'}
)

cond_widget = widgets.Dropdown(
    options=CONDITIONS,
    description='Condition:',
    style={'description_width': '100px'}
)

tesla_widget = widgets.RadioButtons(
    options=[1.5, 3],
    description='Tesla:',
    style={'description_width': '100px'}
)

divider_widget = widgets.Dropdown(
    options={'raw_': 'raw_', 'br_': 'br_', 'Br_': 'Br_'},
    description='Divider:',
    style={'description_width': '100px'}
)

print("Select Parameters:")
print()
display(seq_widget)
display(cond_widget)
display(tesla_widget)
display(divider_widget)

print("\nCurrent Selection:")
print(f"  Sequence: {seq_widget.value}")
print(f"  Condition: {cond_widget.value}")
print(f"  Tesla: {tesla_widget.value}")
print(f"  Divider: {divider_widget.value}")

## 6. Workflow Selection

Choose which pipeline step to execute.

In [ ]:
workflow_widget = widgets.ToggleButtons(
    options={
        'Move Preprocessed': 'move_preprocessed',
        'Move to Preprocess': 'move2preprocess',
        'Move to Convert': 'move2convert',
        'Move Converted': 'moveConverted',
        'Free Move': 'freemove',
    },
    description='Workflow:',
    style={'description_width': '100px'}
)

display(workflow_widget)
print(f"\nSelected: {workflow_widget.value}")

## 7. Workflow Descriptions

Understand what each workflow does before executing.

In [ ]:
workflows_info = {
    'move_preprocessed': {
        'name': 'Move Preprocessed Files',
        'description': 'Move already preprocessed files from old location to preprocessed folder',
        'input': 'preprocessed_old/ (or custom path)',
        'output': 'preprocessed/{seq}/{cond}/',
        'notes': 'Requires metadata to match files with subjects'
    },
    'move2preprocess': {
        'name': 'Move to Preprocess Queue',
        'description': 'Move raw NIFTI files that need preprocessing',
        'input': '3T/{seq}/{cond}/',
        'output': 'TempData/{seq}/{cond}/{subject}-{series}-{image}/',
        'notes': 'Prepares files for preprocessing, organized by subject'
    },
    'move2convert': {
        'name': 'Move to Conversion Queue',
        'description': 'Move DICOM files to conversion folder',
        'input': 'DICOM/{seq}/{cond}/',
        'output': '2convert/{seq}/{cond}/{subject}-{series}_{image}/',
        'notes': 'Prepares for DICOM to NIfTI conversion'
    },
    'moveConverted': {
        'name': 'Move Converted Files',
        'description': 'Move converted NIfTI files to preprocessed folder',
        'input': 'Converted/{seq}/{cond}/',
        'output': 'preprocessed/{seq}/{cond}/',
        'notes': 'Final step after DICOM conversion'
    },
    'freemove': {
        'name': 'Free Move (Pattern-based)',
        'description': 'Flexible file moving based on glob patterns',
        'input': 'Custom path with glob pattern',
        'output': 'Target directory with same structure',
        'notes': 'Most flexible, useful for custom workflows'
    },
}

def display_workflow_info(workflow_name):
    info = workflows_info.get(workflow_name, {})
    print(f"\n{'='*70}")
    print(f"Workflow: {info.get('name', 'Unknown')}")
    print(f"{'='*70}")
    print(f"\nDescription:\n  {info.get('description', 'N/A')}")
    print(f"\nInput Directory:\n  {info.get('input', 'N/A')}")
    print(f"\nOutput Directory:\n  {info.get('output', 'N/A')}")
    print(f"\nNotes:\n  {info.get('notes', 'N/A')}")
    print(f"{'='*70}\n")

display_workflow_info(workflow_widget.value)

## 8. Execute Workflow

Run the selected workflow with chosen parameters.

In [ ]:
execute_button = widgets.Button(
    description='▶ Execute Workflow',
    button_style='success',
    tooltip='Run the selected workflow'
)

execution_output = widgets.Output()

def on_execute_clicked(b):
    with execution_output:
        clear_output()
        try:
            # Check metadata is loaded
            if 'meta_df' not in globals():
                print("⚠ Error: Please load metadata first (Section 4)")
                return
            
            seq = seq_widget.value
            cond = cond_widget.value
            tesla = tesla_widget.value
            divider = divider_widget.value
            workflow = workflow_widget.value
            
            print(f"\n{'='*70}")
            print(f"Executing: {workflows_info[workflow]['name']}")
            print(f"{'='*70}")
            print(f"\nParameters:")
            print(f"  Sequence: {seq}")
            print(f"  Condition: {cond}")
            print(f"  Tesla: {tesla}T")
            print(f"  Divider: {divider}")
            print(f"\nMetadata: {meta_df.shape[0]} records loaded")
            print(f"\nProcessing...\n")
            
            # Execute workflow
            if workflow == 'move_preprocessed':
                meta_dict, processed_indices = movePreprocessed(
                    meta_df, "./preprocessed_old", seq, cond, tesla, divider
                )
                print(f"\n✓ Complete!")
                print(f"  Unprocessed records: {len(meta_dict['Subject'])}")
                
            elif workflow == 'move2preprocess':
                count = move2preprocess(meta_df, seq, cond, tesla, divider)
                print(f"\n✓ Complete!")
                print(f"  Files moved: {count}")
                
            elif workflow == 'move2convert':
                count = move2convert(meta_df, seq, cond, tesla, divider)
                print(f"\n✓ Complete!")
                print(f"  Files moved: {count}")
                
            elif workflow == 'moveConverted':
                count = moveConverted(meta_df, seq, cond, tesla, divider)
                print(f"\n✓ Complete!")
                print(f"  Files moved: {count}")
                
            elif workflow == 'freemove':
                print("\n⚠ Free move requires source/target paths")
                print("  Use: freemove(source, target, seq, cond, tesla, file_format)")
            
            print(f"\n{'='*70}\n")
            
        except Exception as e:
            print(f"✗ Error: {e}")
            import traceback
            traceback.print_exc()

execute_button.on_click(on_execute_clicked)

display(execute_button)
display(execution_output)

## 9. Metadata Filtering & Visualization

Explore and filter metadata interactively.

In [ ]:
if 'meta_df' in globals():
    print(f"Metadata Overview:")
    print(f"  Total Records: {len(meta_df)}")
    print(f"  Columns: {list(meta_df.columns)}")
    print(f"\nUnique Values:")
    
    if 'Group' in meta_df.columns:
        print(f"  Groups: {meta_df['Group'].unique().tolist()}")
    if 'Sex' in meta_df.columns:
        print(f"  Sex: {meta_df['Sex'].unique().tolist()}")
    if 'Modality' in meta_df.columns:
        print(f"  Modalities: {meta_df['Modality'].unique().tolist()}")
    
    # Create filter widgets
    print(f"\n\nFilter Metadata:")
    
    filter_group = widgets.Dropdown(
        options=['All'] + meta_df['Group'].unique().tolist() if 'Group' in meta_df.columns else ['All'],
        description='Group:',
        style={'description_width': '100px'}
    )
    
    filter_sex = widgets.Dropdown(
        options=['All'] + meta_df['Sex'].unique().tolist() if 'Sex' in meta_df.columns else ['All'],
        description='Sex:',
        style={'description_width': '100px'}
    )
    
    filter_button = widgets.Button(description='Apply Filter', button_style='warning')
    filter_output = widgets.Output()
    
    def on_filter_clicked(b):
        with filter_output:
            clear_output()
            filtered = meta_df.copy()
            
            if filter_group.value != 'All':
                filtered = filtered[filtered['Group'] == filter_group.value]
            if filter_sex.value != 'All':
                filtered = filtered[filtered['Sex'] == filter_sex.value]
            
            print(f"\nFiltered Results: {len(filtered)} records")
            display(filtered[['Subject', 'Group', 'Sex', 'Age', 'Modality', 'Visit']].head(10))
    
    filter_button.on_click(on_filter_clicked)
    
    display(filter_group)
    display(filter_sex)
    display(filter_button)
    display(filter_output)
else:
    print("⚠ Please load metadata first (Section 4)")

## 10. Refresh Status

Check pipeline status after running workflows.

In [ ]:
refresh_button = widgets.Button(description='🔄 Refresh Status', button_style='info')
refresh_output = widgets.Output()

def on_refresh_clicked(b):
    with refresh_output:
        clear_output()
        summary = get_directory_summary(str(BASE_DIR))
        print("\n" + "="*70)
        print("Updated Pipeline Status")
        print("="*70)
        print("\nFile Counts by Directory:")
        for name, count in summary.items():
            print(f"  {name}: {count}")
        print("="*70 + "\n")

refresh_button.on_click(on_refresh_clicked)

display(refresh_button)
display(refresh_output)

## 11. Advanced: Custom Function Calls

For advanced users: Call functions directly with custom parameters.

In [ ]:
# Example: Direct function calls

# Example 1: Export metadata to CSV
# from libs.metadata import exportCSV
# meta_dict = {...}  # your metadata dict
# exportCSV(meta_dict, "my_metadata", output_dir="./TempMeta/")

# Example 2: Filter metadata
# from libs.metadata import filterMetadata
# filtered_df = filterMetadata(meta_df, Group='AD', Sex='M')

# Example 3: Create combined ID strings for matching
# from libs.metadata import createMetaCombinedString
# combined_ids = createMetaCombinedString(meta_df)
# print(combined_ids[:5])  # First 5 combined IDs

print("Uncomment and modify examples above to use custom function calls.")
print("\nAll functions from libs are available:")
print("  • libs.file_operations")
print("  • libs.metadata")
print("  • libs.utils")
print("  • libs.config")
print("  • libs.logging")

## 12. Troubleshooting & Tips

Common issues and solutions.

In [ ]:
troubleshooting = """
COMMON ISSUES & SOLUTIONS
========================

1. "Metadata file not found"
   → Ensure metadata CSV files are in TempMeta/ directory
   → Run: ensure_output_directories(str(BASE_DIR))

2. "No files found during move operation"
   → Check source directory structure matches expectations
   → Verify directory validation in Section 2 passes
   → Check filename patterns (raw_, br_, Br_)

3. "ModuleNotFoundError: No module named 'libs'"
   → Restart the kernel
   → Ensure notebook is in ADNI-processing/ directory
   → Check sys.path.insert in Section 1

4. "Unmatched metadata records"
   → Metadata combined IDs don't match filenames
   → Check divider parameter (raw_, br_, Br_)
   → Verify filename format: ADNI_[ID]_MR[divider][SERIES]_I[IMAGE]

5. "Permission denied when moving files"
   → Ensure target directories are writable
   → Check Windows file permissions
   → Try running with administrator privileges

FOR BATCH PROCESSING:
→ Use shell scripts in scripts/ directory instead
→ run_pipeline.sh for fully automated pipeline

FOR DEBUGGING:
→ Add breakpoints and check variables
→ Use print_pipeline_status() to inspect state
→ Check logs in outputs/logs/ directory
"""

print(troubleshooting)